In [1]:
import tensorflow as tf

D:\Anaconda\Initial\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
#attention
def attention(self,prev_state,enc_outputs):
    #prev_state即前一个横向状态输出信息，即为S[i-1]
    #enc_outputs编码过程的每个纵向特征信息输出
    e_i = []
    c_i = []
    for output in enc_outputs:
        atten_hidden = tf.tanh(tf.add(tf.matmul(prev_state,self.attention_W),tf.matmul(output,self.attention_U)))
        # atten_hidden = tanh(S[i-1]*W+h*U)    tf.matmul为矩阵相乘
        e_i_i = tf.matmul(atten_hidden,self.attention_V)
        # e_i_i = V * atten_hidden
        e_i.append(e_i_i)
        #e_i: [[],[],...,[]]   内部每个[]都是一个e_i_i
    e_i = tf.concat(e_i,axis=1)  # 将e_i里的每一个e_i_i按第1维度拼接
    alpha_i = tf.nn.softmax(e_i) # 对每个e_i进行softMax概率转换，此时得到的是一个向量
    alpha_i = tf.split(alpha_i,self.num_steps,1) # 对第一个维度拆成num_step个,num_steps即encoder句子长度，为了后面与每个h进行对应相乘相加
    for alpha_i_i,output in zip(alpha_i,enc_outputs):#即拆成每个alpha和h
        c_i_i = tf.multiply(alpha_i_i,output)  #tf.multiply为对应元素相乘相加，即每个a[i]*h[i],然后累加
        c_i.append(c_i_i) #变成了多个元素
    c_i = tf.reshape(tf.concat(c_i,axis=1),[-1,self.num_steps,self.hidden_dim*2]) #concat成一个向量，然后reshape，这里hidden*2表明是双向的
    #-1是batch_size，numsteps是encoder的句子长度，hidden_dim*2是隐藏元维度数
    c_i = tf.reduce_sum(c_i,1) #将num_step的维度合并成1才能在后面和s[i-1],y[i]进行拼接
    return c_i

In [ ]:
#decoder
def decoder(self,cell,init_state,encoder_outputs,loop_function=None):
    outputs = []
    prev = None
    state = init_state
    for i,inp in enumerate(self.decoder_input_emb):
        #decoder_inputs_emb是tf.placeholder
#         if loop_function is not None ad prev is not None:
#             with tf.variable_scope("loop_function",reuse=True):
#                 inp = loop_function(prev,i)
#         if i >0:
#             tf.get_variable_scope(),reuse_variables()
        c_i = self.attention(state,encoder_outputs) #得到C_i
        inp = tf.concat([inp,c_i],axis=1)
        output,state = cell(inp,state) #原本没有attention的是decoder_input和state作为输入
        #即放到lstm里或gru里，output即纵向上的特征信息输出，state即横向上的状态信息输出
        outputs.append(output) #只要纵向上的特征信息输出
        if loop_function is not None: #循环未结束，将当前的纵向输出给prev，用来下一次表示“上一个纵向特征信息输出”
            prev = output
    return outputs
